In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_cvc5 = pd.read_csv('../benchmark/sygus_solver_cvc5_results_new.csv')
df_cvc5['config'] = (df_cvc5['config']
                .str.replace('--tlimit=30000', '')
                .str.replace('--sygus-enum=', '')
                .str.replace('--sygus-si=', '_single_invocation_')
                .str.replace('--', '_')
                .str.replace('-', '_')
                .str.replace('=', '_')
                .str.replace(' ', '')
                )
df_cvc5['file'] = df_cvc5['file'].str.replace('/data/Documents/DataScience/Courses/Dissertation/problems/sygus_comp_2019_clia_track/', '').str.replace('.sl','').str.strip()


df_pysynthlab = pd.read_csv('../benchmark/sygus_solver_pysynthlab_results_new.csv')
df_pysynthlab['run_datetime'] = pd.to_datetime(df_pysynthlab['run_datetime'])
df_pysynthlab['solver'] = 'PySynthLab'
df_pysynthlab['config'] = df_pysynthlab['config'].str.split('/').str[-1].str.replace('.yaml', '').str.replace('benchmark_','')
df_pysynthlab['file'] = df_pysynthlab['file'].str.split('/').str[-1].str.replace('.sl', '')


competition_df = pd.read_csv('../competition_results.csv')